In [1]:
#导入库与数据集
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree

raw=pd.read_csv('exp3.csv')
label=pd.read_csv('label.csv')
label.head()

,index,gameDuration
0,20000,1671
1,20001,1227
2,20002,1458
3,20003,1317
4,20004,2091


In [2]:
def geten(df):
    en=pd.DataFrame()
    
    raw1=df.iloc[:,12:43]
    raw2=df.iloc[:,44:78]
    raw1['t1_kills']=raw1['player1_kills']+raw1['player2_kills']+raw1['player3_kills']+raw1['player4_kills']+raw1['player5_kills']
    raw1['t1_deaths']=raw1['player1_deaths']+raw1['player2_deaths']+raw1['player3_deaths']+raw1['player4_deaths']+raw1['player5_deaths']
    raw1['t1_assists']=raw1['player1_assists']+raw1['player2_assists']+raw1['player3_assists']+raw1['player4_assists']+raw1['player5_assists']
    raw2['t2_kills']=raw2['player6_kills']+raw2['player7_kills']+raw2['player8_kills']+raw2['player9_kills']+raw2['player10_kills']
    raw2['t2_deaths']=raw2['player6_deaths']+raw2['player7_deaths']+raw2['player8_deaths']+raw2['player9_deaths']+raw2['player10_deaths']
    raw2['t2_assists']=raw2['player6_assists']+raw2['player7_assists']+raw2['player8_assists']+raw2['player9_assists']+raw2['player10_assists']
    en['kills'] = raw1['t1_kills']+raw2['t2_kills']
    en['assists'] = raw1['t1_assists']+raw2['t2_assists']
    en['deaths'] = raw1['t1_deaths']+raw2['t2_deaths']
    
    raw1=df.loc[:,['player1_goldEarned','player2_goldEarned','player3_goldEarned','player4_goldEarned','player5_goldEarned']]
    raw2=df.loc[:,['player6_goldEarned','player7_goldEarned','player8_goldEarned','player9_goldEarned','player10_goldEarned']]
    raw1['gold1'] = raw1.apply(lambda x: x.sum(), axis=1)
    raw2['gold2'] = raw2.apply(lambda x: x.sum(), axis=1)
    en['gold'] = raw1['gold1']+raw2['gold2']
    
    #en['gameDuration'] = df['gameDuration']                                                                                                                        
    return en    

In [18]:
#初始化DataFrame文件
df=raw.loc[20000:]
df=df.reset_index(drop=True)
df=pd.concat([df,label],axis=1)
df=df.reindex(np.random.permutation(len(df))) #随机打乱
df.head()

,index,queueId,mapId,seasonId,team1_win,team1_firstBlood,team1_firstTower,team1_firstInhibitor,team1_firstBaron,team1_firstDragon,...,player9_lane,player10_championId,player10_kills,player10_deaths,player10_assists,player10_goldEarned,player10_role,player10_lane,index,gameDuration
42565,62565,420,11,13,1,True,True,False,False,True,...,NONE,67,0,2,0,4668,DUO,NONE,62565,914
47601,67601,430,11,13,0,True,False,False,False,True,...,JUNGLE,39,8,2,3,11218,SOLO,MIDDLE,67601,1376
36956,56956,420,11,13,0,False,False,False,False,False,...,NONE,412,2,0,7,5098,DUO_SUPPORT,NONE,56956,914
22087,42087,430,11,13,1,False,True,False,False,True,...,NONE,55,1,4,2,4955,DUO_SUPPORT,NONE,42087,935
31275,51275,420,11,13,0,False,False,False,False,False,...,MIDDLE,122,2,0,2,9797,SOLO,TOP,51275,1311


In [19]:
train=geten(df)
train.head()
train=(train-train.min())/(train.max()-train.min())
train['gameDuration']=df['gameDuration']
target=df['gameDuration']
#target=((target/45).astype('int')+1)*45
target.head()

42565     914
47601    1376
36956     914
22087     935
31275    1311
Name: gameDuration, dtype: int64

In [20]:
x_train=train.iloc[0:48000,0:4]
x_test=train.iloc[48001:,0:4]

y_train=target.iloc[0:48000]

y_test=train.iloc[48001:,4]
x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values
print (x_train)
print (y_train)
print (x_test)
print (y_test)

[[0.09659091 0.04846939 0.0960452  0.18664129]
 [0.26136364 0.14030612 0.25988701 0.332978  ]
 [0.11931818 0.04336735 0.11864407 0.18003384]
 ...
 [0.27272727 0.19387755 0.27118644 0.47886633]
 [0.32386364 0.28061224 0.3220339  0.50056261]
 [0.13068182 0.06122449 0.1299435  0.19717005]]
[ 914 1376  914 ... 2039 2056 1003]
[[0.22727273 0.11989796 0.23163842 0.27725888]
 [0.15909091 0.0994898  0.15819209 0.28319374]
 [0.11363636 0.06122449 0.11864407 0.21917513]
 ...
 [0.15909091 0.09438776 0.15819209 0.26548223]
 [0.375      0.24234694 0.37853107 0.44439509]
 [0.34659091 0.36989796 0.34463277 0.56651015]]
[1248 1309 1043 ... 1232 1762 2431]


In [21]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x_train,y_train)
pre=clf.predict(x_test)
err=metrics.mean_squared_error(pre,y_test)
print(err)

24668.776481373447


In [22]:
#检测合格率
n=0
for i in range(len(pre)):
    if abs(pre[i]-y_test[i])<100:
        n=n+1
print('预测合格率为：',n/len(pre))

预测合格率为： 0.7678973247770647


In [8]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier

In [23]:
train=geten(df)
train=(train-train.min())/(train.max()-train.min())
train['gameDuration']=df['gameDuration']
target=df['gameDuration']

In [24]:
x_train=train.iloc[0:48000,0:4]
x_test=train.iloc[48001:,0:4]
y_train=target.iloc[0:48000]
y_test=train.iloc[48001:,4]
x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values

In [38]:
clf = MLPRegressor(solver='adam', alpha=1e-5,max_iter=1500,learning_rate_init=0.005,
                 hidden_layer_sizes=(8,),random_state=1)

clf.fit(x_train,y_train)

MLPRegressor(alpha=1e-05, hidden_layer_sizes=(8,), learning_rate_init=0.005,
             max_iter=1500, random_state=1)

In [39]:
pre=clf.predict(x_test)
err=metrics.mean_squared_error(pre,y_test)
print(pre,err)

[1184.91415253 1303.64160819 1073.08453765 ... 1225.1635191  1741.45968532
 2350.14716721] 18261.572422111127


In [40]:
#检测合格率
n=0
for i in range(len(pre)):
    if abs(pre[i]-y_test[i])<100:
        n=n+1
print(n/len(pre))

0.8611550962580214
